In [ ]:
import os
import requests
from bs4 import BeautifulSoup

BASE_URL = "https://papers.nips.cc/"
YEARS = range(2024, 2019, -1)
DOWNLOAD_DIR = r"C:\\Users\\Qadri Laptop\\NeurIPS_Papers"

os.makedirs(DOWNLOAD_DIR, exist_ok=True)

def get_paper_links(year):
    response = requests.get(f"{BASE_URL}paper_files/paper/{year}")
    if response.status_code != 200:
        return []
    
    soup = BeautifulSoup(response.text, "html.parser")
    return [BASE_URL.rstrip("/") + link.get("href") for link in soup.find_all("a")
            if link.get("href", "").startswith("/paper_files/paper/") and link.get("href").endswith(".html")]

def download_paper(paper_url, year):
    response = requests.get(paper_url)
    if response.status_code != 200:
        return

    soup = BeautifulSoup(response.text, "html.parser")
    pdf_link = next((BASE_URL.rstrip("/") + link.get("href")
                    for link in soup.find_all("a") if "Paper" in link.text), None)
    
    if not pdf_link or os.path.exists(file_path := os.path.join(DOWNLOAD_DIR, str(year), pdf_link.split("/")[-1])):
        return

    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    with open(file_path, "wb") as file:
        for chunk in requests.get(pdf_link, stream=True).iter_content(1024):
            file.write(chunk)
    print(f"Downloaded: {file_path}")

if __name__ == "__main__":
    for year in YEARS:
        for paper_url in get_paper_links(year):
            download_paper(paper_url, year)
    print("Scraping completed!")
